In [1]:
#| default_exp implementation.monitoring

# Monitoring Source Code Project

This implementation wraps `feature.codeengine` and `feature.enterprise_apps` to sync user generated code to an appdb collection

- [x] - implement a class `DocumentSource` for capturing source code.
    - [x] classmethod for creating `DocumentSource.from_codeengine_package_version`
        - retrieves most recent version of codeengine package
    
    - [x] classmethod for creating `DocumentSource.from_source_file` 
        - [ ] workflow for enterprise apps? - one enterprise app can have multiple source_files.  Will need to consider how to structure documents in collection.  
            - in this sample implementation apps shared with the user will be downloaded to the folder TEST
            - [ ] create workflow for auto sharing all content with the user
            
        - [ ] workflow for jupyter notebooks? - currently cannot properly automate as unable to retrieve content from jupyter workspaces without the authentication token which must be scraped from the UI
    
- [ ] - write to AppDb Collection
    - Not started yet 
        


In [2]:
#| exporti

from dataclasses import dataclass, field

from typing import List
import datetime as dt

import mbison.client.core as dmda
import mbison.feature.users as dmdu

import mbison.feature.appdb as dmdb
import mbison.feature.cards as dmdc
import mbison.feature.codeengine as dmce
import mbison.feature.enterprise_apps as dmap

import mbison.utils as dmut


In [3]:
#| hide
import nbdev

import os
from pprint import pprint

In [4]:
auth = dmda.DomoAuth(
    domo_instance=os.environ["DOMO_INSTANCE"],
    access_token=os.environ["DOMO_ACCESS_TOKEN"],
)
auth

DomoAuth(domo_instance='domo-community', username=None)

In [7]:
card_id = '577316875'

query = {
        "includeCardTypeClause": True,
        "cardTypes": ["domoapp",
                      "mason","custom"
                      ],
        "ascending": True,
        "orderBy": "cardTitle",
    }

# res = search_cards(auth = auth, query = query)
# cards = res.response

In [14]:
import mbison.feature.cards as dmac
import mbison.feature.appdb as dmdb


def get_source_code(card_id, auth, debug_api : bool = False):

    res = dmac.get_card_by_id(card_id = card_id, auth = auth, debug_api= debug_api)
    
    datastore_id = res.response['domoapp']['id']
    datastore_id

    res = dmdb.get_collections(auth=auth, debug_api= debug_api,
                    datastore_id= datastore_id
                    )
    collection = res.response[0]
    
    res = dmdb.query_collection_documents(auth=auth,
                                debug_api= debug_api,
                                collection_id = collection['id'],query = {}
                                )
    
    documents = res.response
    documents

    return documents[0]
    
get_source_code(card_id = card_id, auth = auth, debug_api = False)

{'id': '3ed44deb-ce70-4e88-9fc7-985cb7f21435',
 'datastoreId': 'c3314265-965a-46b2-9e21-5ad27fd1101b',
 'collectionId': '475cc3b8-4318-406a-8070-c023bf0b9152',
 'syncRequired': True,
 'owner': '1893952720',
 'createdBy': '1893952720',
 'createdOn': '2023-05-05T21:54:30.178Z',
 'updatedOn': '2023-11-02T15:18:02.629Z',
 'updatedBy': '1893952720',
 'content': {'htmlBlank': {'js': "// DDX Bricks Wiki - See https://developer.domo.com/docs/ddx-bricks/getting-started-using-ddx-bricks\n// for tips on getting started, linking to Domo data and debugging your app\n \n//Available globals\nvar domo = window.domo; // For more on domo.js: https://developer.domo.com/docs/dev-studio-guides/domo-js#domo.get\nvar datasets = window.datasets;\n\n//Step 1. Select your dataset(s) from the button in the bottom left corner\n\n\n\n//Step 2. Query your dataset(s): https://developer.domo.com/docs/dev-studio-references/data-api\nvar fields = ['state', 'revenue'];\nvar groupby = ['state'];\nvar query = `/data/v1/${

In [ ]:
# | exports


class SourceCode_Enum:
    """captures metadata about where source came from"""
    CODE_ENGINE = "Code Engine"
    ENTERPRISE_APP = "Enterprise App"


@dataclass
class DocumentSource:
    environment: SourceCode_Enum
    id: str
    name: str
    design_id: str
    design_name: str
    log_dt: dt.datetime
    code: str
    version: str
    last_modified_dt: dt.datetime
    domo_instance: str
    owner: str

    def __eq__(self, other):
        if not self.__class__.__name__ == other.__class__.__name__:
            return False

        return self.id == other.id and self.environment == other.environment

    @classmethod
    def from_codeengine_package_version(
        cls, dce_package: dmce.DomoCodeEngine_Package, log_dt: dt.datetime = None
    ):

        log_dt = log_dt or dt.datetime.now()

        dce_current_version = dce_package.get_current_version()

        # from pprint import pprint
        # pprint({"version": dce_current_version,
        #        "owner" : dce_package.owner})

        owner = None
        try:
            owner = dmdu.DomoUser.get_by_id(
                auth=dce_package.auth, user_id=dce_package.owner
            )

        except dmdu.User_API_Exception as e:
            print(e)

        return cls(
            environment=SourceCode_Enum.CODE_ENGINE,
            id=dce_package.id,
            name=dce_package.name,
            design_id=dce_package.id,
            design_name=dce_package.name,
            log_dt=log_dt,
            code=dce_current_version.code,
            version=dce_current_version.version,
            last_modified_dt=dce_package.updated_on_dt,
            domo_instance=dce_package.auth.domo_instance,
            owner=owner,
        )

    @classmethod
    def from_source_file(
        cls,
        file_path,
        design_id,
        design_name,
        domo_instance: str,
        owner: dmdu.DomoUser,
        log_dt: dt.datetime = None,
        version: str = None,
        last_modified_dt: dt.datetime = None,
    ):

        with open(file_path, "r") as f:
            code = f.read()

        log_dt = log_dt or dt.datetime.now()

        return cls(
            id=file_path,
            name=os.path.split(file_path)[-1],
            design_id=design_id,
            design_name=design_name,
            log_dt=log_dt,
            code=code,
            version=version,
            last_modified_dt=last_modified_dt,
            domo_instance=domo_instance,
            owner=owner,
            environment=SourceCode_Enum.ENTERPRISE_APP,
        )

        # custom_app, design_version, appdb_collection):
        # document["ID"] = custom_app["id"]
        # document["Name"] = custom_app["name"]
        # document["Published Date"] = custom_app["updatedDate"]


@dataclass
class DocumentSources:
    auth: dmda.DomoAuth
    documents: List[DocumentSource] = field(default_factory=lambda: [])

    log_dt: dt.datetime = dt.datetime.now()

    def get_codeengine(self):
        domo_codenegine_packages = dmce.DomoCodeEngine_Packages(
            auth=self.auth
        ).get_packages()

        documents = [
            DocumentSource.from_codeengine_package_version(
                dce_package=dce_package, log_dt=self.log_dt
            )
            for dce_package in domo_codenegine_packages
        ]

        [self.add_document(document) for document in documents]

        return self.documents

    @staticmethod
    def _download_custom_app_sourcecode(app, download_folder, debug_api: bool = False):
            zip_name = "_package.zip"
            _download_folder = f"{download_folder}/{app.name}/{app.current_version}"

            try:
                res = app.get_source_code(download_folder=_download_folder, file_name=zip_name, debug_api = debug_api)
                
                zip_file_path=os.path.join(
                        _download_folder, zip_name
                    )
                
                path_ls = dmut.download_zip(
                    output_folder=_download_folder,
                    zip_file_path=zip_file_path,
                )

                res.response = path_ls

                return res
            
            except dmap.App_API_Exception as e:
                print(e)
                return False


    def get_custom_apps(self, download_folder="./EXPORT", debug_api : bool = False):
        domo_apps = dmap.DomoEnterpriseApps(auth=self.auth).get_apps()

        return [self._download_custom_app_sourcecode(app = app, download_folder = download_folder, debug_api = debug_api) for app in domo_apps]            


    def add_document(self, document: DocumentSource):
        if document in self.documents:
            return False

        self.documents.append(document)
        return True

In [ ]:
document_sources = DocumentSources(auth= auth)

res_ls = document_sources.get_custom_apps(download_folder= '../../TEST/' ,debug_api = False)
[res for res in res_ls if res ][0:5]

In [ ]:
document_sources = DocumentSources(auth= auth)

document_sources.get_codeengine()

In [ ]:
document_sources.documents[0]

In [ ]:
document_sources.documents[0].__dict__

In [ ]:
#| hide


nbdev.nbdev_export('./monitoring.ipynb')